# Loan Sales Prediction - Data Preparation

This notebook handles:
1. Loading raw data from Excel
2. Data cleaning and preprocessing
3. Feature engineering
4. Converting to ML-ready format
5. Saving processed data for model training

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries imported successfully")

## 2. Load Raw Data

In [ ]:
# Define paths
DATA_DIR = Path('../notebooks/data')
RAW_DATA_PATH = DATA_DIR / 'loan_sales.xlsx'
PROCESSED_DATA_PATH = DATA_DIR / 'ml_ready_data.csv'
DASHBOARD_DATA_PATH = Path('../analytics-dashboard/public/data/ml_ready_data.csv')

# Create directories if they don't exist
DATA_DIR.mkdir(parents=True, exist_ok=True)
DASHBOARD_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"📂 Data directory: {DATA_DIR.absolute()}")
print(f"📂 Dashboard directory: {DASHBOARD_DATA_PATH.parent.absolute()}")

In [ ]:
# Load the Excel file
if not RAW_DATA_PATH.exists():
    raise FileNotFoundError(f"❌ Data file not found: {RAW_DATA_PATH}")

df_raw = pd.read_excel(RAW_DATA_PATH)

print(f"✅ Data loaded successfully")
print(f"📊 Shape: {df_raw.shape}")
print(f"\n📋 First few rows:")
df_raw.head()

## 3. Data Exploration

In [ ]:
# Display basic information
print("📊 Dataset Information:")
print(f"  Rows: {df_raw.shape[0]}")
print(f"  Columns: {df_raw.shape[1]}")
print(f"\n📋 Column Names:")
for i, col in enumerate(df_raw.columns, 1):
    print(f"  {i}. {col}")

In [ ]:
# Check data types
print("📊 Data Types:")
df_raw.dtypes

In [ ]:
# Check for missing values
print("🔍 Missing Values:")
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

In [ ]:
# Display summary statistics
print("📊 Summary Statistics:")
df_raw.describe()

## 4. Parse Quarter Information

Convert quarter format from Roman numerals (e.g., "2020 I") to numeric format

In [ ]:
# Create a working copy
df = df_raw.copy()

# Parse Rüblər column to extract Year and Quarter
roman_to_int = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}

def parse_quarter(quarter_str):
    """Parse quarter string like '2020 I' to year and quarter number"""
    if pd.isna(quarter_str):
        return None, None
    
    parts = str(quarter_str).strip().split()
    if len(parts) != 2:
        return None, None
    
    year = int(parts[0])
    quarter_roman = parts[1].strip()
    quarter = roman_to_int.get(quarter_roman)
    
    return year, quarter

# Apply parsing
df[['Year', 'Quarter']] = df['Rüblər'].apply(
    lambda x: pd.Series(parse_quarter(x))
)

print("✅ Quarter parsing completed")
print(f"\n📅 Date Range:")
print(f"  First Quarter: {df['Year'].min()}-Q{df['Quarter'].min()}")
print(f"  Last Quarter: {df['Year'].max()}-Q{df['Quarter'].max()}")
print(f"\n📋 Sample of parsed data:")
df[['Rüblər', 'Year', 'Quarter']].head(10)

## 5. Feature Engineering

In [ ]:
# Create time-based features
df['Time_Index'] = range(len(df))
df['Quarter_Sin'] = np.sin(2 * np.pi * df['Quarter'] / 4)
df['Quarter_Cos'] = np.cos(2 * np.pi * df['Quarter'] / 4)

print("✅ Time-based features created")
print("\n📊 New Features:")
print("  - Time_Index: Sequential time index")
print("  - Quarter_Sin: Sine encoding of quarter (captures seasonality)")
print("  - Quarter_Cos: Cosine encoding of quarter (captures seasonality)")

df[['Year', 'Quarter', 'Time_Index', 'Quarter_Sin', 'Quarter_Cos']].head(10)

In [ ]:
# Create lag features for target variable (Nağd_pul_kredit_satışı)
target_col = 'Nağd_pul_kredit_satışı'

if target_col in df.columns:
    # Lag features
    df['Lag_1'] = df[target_col].shift(1)
    df['Lag_2'] = df[target_col].shift(2)
    df['Lag_3'] = df[target_col].shift(3)
    df['Lag_4'] = df[target_col].shift(4)
    
    # Rolling statistics
    df['Rolling_Mean_2'] = df[target_col].rolling(window=2).mean()
    df['Rolling_Mean_3'] = df[target_col].rolling(window=3).mean()
    df['Rolling_Mean_4'] = df[target_col].rolling(window=4).mean()
    
    df['Rolling_Std_2'] = df[target_col].rolling(window=2).std()
    df['Rolling_Std_3'] = df[target_col].rolling(window=3).std()
    df['Rolling_Std_4'] = df[target_col].rolling(window=4).std()
    
    # Difference features
    df['Diff_1'] = df[target_col].diff(1)
    df['Diff_4'] = df[target_col].diff(4)  # Year-over-year change
    
    print("✅ Lag and rolling features created")
    print("\n📊 New Features:")
    print("  Lag Features: Lag_1, Lag_2, Lag_3, Lag_4")
    print("  Rolling Means: Rolling_Mean_2, Rolling_Mean_3, Rolling_Mean_4")
    print("  Rolling Std: Rolling_Std_2, Rolling_Std_3, Rolling_Std_4")
    print("  Differences: Diff_1 (quarter-over-quarter), Diff_4 (year-over-year)")
else:
    print(f"⚠️ Warning: Target column '{target_col}' not found")

## 6. Data Quality Checks

In [ ]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"🔍 Duplicate Rows: {duplicates}")

if duplicates > 0:
    print("\n⚠️ Removing duplicates...")
    df = df.drop_duplicates()
    print(f"✅ Duplicates removed. New shape: {df.shape}")

In [ ]:
# Check for infinite values
numeric_cols = df.select_dtypes(include=[np.number]).columns
inf_counts = {}

for col in numeric_cols:
    inf_count = np.isinf(df[col]).sum()
    if inf_count > 0:
        inf_counts[col] = inf_count

if inf_counts:
    print("⚠️ Infinite values found:")
    for col, count in inf_counts.items():
        print(f"  {col}: {count}")
    
    # Replace infinite values with NaN
    df = df.replace([np.inf, -np.inf], np.nan)
    print("\n✅ Infinite values replaced with NaN")
else:
    print("✅ No infinite values found")

In [ ]:
# Display updated missing values after feature engineering
print("🔍 Missing Values After Feature Engineering:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
missing_summary = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_summary) > 0:
    print(missing_summary)
else:
    print("✅ No missing values")

## 7. Visualize Target Variable

In [ ]:
# Plot target variable over time
if target_col in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Time series plot
    ax1 = axes[0, 0]
    df_valid = df[df[target_col].notna()]
    ax1.plot(df_valid['Time_Index'], df_valid[target_col], marker='o', linewidth=2, markersize=6)
    ax1.set_title('Loan Sales Over Time', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Time Index')
    ax1.set_ylabel('Loan Sales (AZN)')
    ax1.grid(True, alpha=0.3)
    
    # Distribution
    ax2 = axes[0, 1]
    ax2.hist(df_valid[target_col], bins=20, edgecolor='black', alpha=0.7)
    ax2.set_title('Distribution of Loan Sales', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Loan Sales (AZN)')
    ax2.set_ylabel('Frequency')
    ax2.grid(True, alpha=0.3)
    
    # Box plot by quarter
    ax3 = axes[1, 0]
    df_valid.boxplot(column=target_col, by='Quarter', ax=ax3)
    ax3.set_title('Loan Sales by Quarter', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Quarter')
    ax3.set_ylabel('Loan Sales (AZN)')
    plt.suptitle('')  # Remove auto-generated title
    
    # Year-over-year growth
    ax4 = axes[1, 1]
    if 'Diff_4' in df.columns:
        df_diff = df[df['Diff_4'].notna()]
        ax4.bar(df_diff['Time_Index'], df_diff['Diff_4'], alpha=0.7)
        ax4.axhline(y=0, color='red', linestyle='--', linewidth=1)
        ax4.set_title('Year-over-Year Change', fontsize=14, fontweight='bold')
        ax4.set_xlabel('Time Index')
        ax4.set_ylabel('YoY Change (AZN)')
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n📊 Target Variable Statistics:")
    print(f"  Mean: {df_valid[target_col].mean():,.2f} AZN")
    print(f"  Median: {df_valid[target_col].median():,.2f} AZN")
    print(f"  Std Dev: {df_valid[target_col].std():,.2f} AZN")
    print(f"  Min: {df_valid[target_col].min():,.2f} AZN")
    print(f"  Max: {df_valid[target_col].max():,.2f} AZN")

## 8. Save Processed Data

In [ ]:
# Save to both locations
print("💾 Saving processed data...\n")

# Save to notebooks/data (for ML training)
df.to_csv(PROCESSED_DATA_PATH, index=False)
print(f"✅ Saved to: {PROCESSED_DATA_PATH}")

# Save to dashboard public folder (for API)
df.to_csv(DASHBOARD_DATA_PATH, index=False)
print(f"✅ Saved to: {DASHBOARD_DATA_PATH}")

print(f"\n📊 Final Dataset:")
print(f"  Shape: {df.shape}")
print(f"  Rows: {df.shape[0]}")
print(f"  Columns: {df.shape[1]}")

In [ ]:
# Display final column list
print("📋 Final Column List:\n")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Display sample of final data
print("📊 Sample of Processed Data:\n")
df.tail(10)

## 9. Data Summary Report

In [ ]:
# Generate comprehensive summary
print("="*80)
print("DATA PREPARATION SUMMARY")
print("="*80)

print("\n📊 DATASET OVERVIEW:")
print(f"  Original Shape: {df_raw.shape}")
print(f"  Final Shape: {df.shape}")
print(f"  New Features Created: {df.shape[1] - df_raw.shape[1]}")

print("\n📅 TIME PERIOD:")
print(f"  Start: {df['Year'].min()}-Q{df['Quarter'].min()}")
print(f"  End: {df['Year'].max()}-Q{df['Quarter'].max()}")
print(f"  Total Quarters: {len(df[df['Year'].notna()])}")

print("\n📈 TARGET VARIABLE (Nağd_pul_kredit_satışı):")
if target_col in df.columns:
    valid_target = df[df[target_col].notna()][target_col]
    print(f"  Valid Records: {len(valid_target)}")
    print(f"  Mean: {valid_target.mean():,.2f} AZN")
    print(f"  Std Dev: {valid_target.std():,.2f} AZN")
    print(f"  Min: {valid_target.min():,.2f} AZN")
    print(f"  Max: {valid_target.max():,.2f} AZN")

print("\n🔧 FEATURE TYPES:")
print(f"  Original Features: {len(df_raw.columns)}")
print(f"  Time Features: 5 (Year, Quarter, Time_Index, Quarter_Sin, Quarter_Cos)")
print(f"  Lag Features: 4 (Lag_1 to Lag_4)")
print(f"  Rolling Features: 6 (Rolling means and stds)")
print(f"  Difference Features: 2 (Diff_1, Diff_4)")
print(f"  Total Features: {len(df.columns)}")

print("\n📁 FILES SAVED:")
print(f"  1. {PROCESSED_DATA_PATH}")
print(f"  2. {DASHBOARD_DATA_PATH}")

print("\n" + "="*80)
print("✅ DATA PREPARATION COMPLETE")
print("="*80)

## Next Steps

1. **Exploratory Data Analysis (EDA)**: Create `02_eda.ipynb` for detailed analysis
2. **Model Training**: Create `03_model_training.ipynb` for ML models
3. **Model Evaluation**: Create `04_model_evaluation.ipynb` for comparing models
4. **Forecasting**: Use the trained models to make predictions